In [ ]:
# Load the CIFAR10 data

# import numpy as np
# import pickle
# import os

# def unpickle(file):
#     with open(file, 'rb') as fo:
#         dict = pickle.load(fo, encoding='bytes')
#     return dict

# data = []
# labels = []
# for i in range(1, 6):
#     filename = os.path.join('cifar-10-python','cifar-10-batches-py', 'data_batch_' + str(i))
#     batch = unpickle(filename)
#     for i in range(len(batch[b'labels'])):
#         if batch[b'labels'][i] == 0 or batch[b'labels'][i] == 1:
#             data.append(batch[b'data'][i])
#             labels.append(batch[b'labels'][i])

# data = np.array(data)
# labels = np.array(labels)
# # print(data.shape)

# # Get the test data
# filename = os.path.join('cifar-10-python','cifar-10-batches-py', 'test_batch')
# batch = unpickle(filename)
# test_data = []
# test_labels = []
# for i in range(len(batch[b'labels'])):
#     if batch[b'labels'][i] == 0 or batch[b'labels'][i] == 1:
#         test_data.append(batch[b'data'][i])
#         test_labels.append(batch[b'labels'][i])

# test_data = np.array(test_data)
# test_labels = np.array(test_labels)
# # print(test_data.shape)

import numpy as np
import pickle
import os

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

data = []
labels = []
for i in range(1,6):
    batch = unpickle(os.path.join('cifar-10-python','cifar-10-batches-py', 'data_batch_' + str(i)))
    data.append(batch[b'data'])
    labels.append(batch[b'labels'])
data = np.concatenate(data)
labels = np.concatenate(labels)

# print(data.shape)

test_batch = unpickle(os.path.join('cifar-10-python','cifar-10-batches-py', 'data_batch_' + str(i)))
test_data = test_batch[b'data']
test_labels = np.array(test_batch[b'labels'])

# print(test_data.shape)


In [ ]:
# Implement the Conv2D layer
class Conv2D:
    def __init__(self, in_channels, out_channels, kernel_size):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        # Initialize the weights
        self.weights = np.random.randn(out_channels, in_channels, kernel_size, kernel_size)
        self.bias = np.random.randn(out_channels, 1)

    def forward(self, x):
        # Implement the forward pass for the Conv2D layer
        # print(x.shape)
        # print(self.weights.shape)
        # print(self.bias.shape)
        self.x = x
        # print(self.x.shape)
        self.output = np.zeros((self.out_channels, x.shape[0] - self.kernel_size + 1, x.shape[1] - self.kernel_size + 1))
        print(self.output.shape)
        for i in range(self.out_channels):
            for j in range(self.x.shape[0] - self.kernel_size + 1):
                for k in range(self.x.shape[1] - self.kernel_size + 1):
                    self.output[i][j][k] = np.sum(self.weights[i] * self.x[j:j + self.kernel_size, k:k + self.kernel_size]) + self.bias[i]
        print(self.output.shape)
        return self.output

    def backward(self, grad_output):
        # Implement the backward pass for the Conv2D layer
        # grad_output is the gradient from the previous layer
        # Return the gradient for the current layer
        self.grad_weights = np.zeros((self.out_channels, self.in_channels, self.kernel_size, self.kernel_size))
        self.grad_bias = np.zeros((self.out_channels, 1))
        self.grad_input = np.zeros((self.in_channels, self.x.shape[0], self.x.shape[1]))
        for i in range(self.out_channels):
            for j in range(self.x.shape[0] - self.kernel_size + 1):
                for k in range(self.x.shape[1] - self.kernel_size + 1):
                    self.grad_weights[i] += grad_output[i][j][k] * self.x[j:j + self.kernel_size, k:k + self.kernel_size]
                    self.grad_bias[i] += grad_output[i][j][k]
                    self.grad_input[:, j:j + self.kernel_size, k:k + self.kernel_size] += grad_output[i][j][k] * self.weights[i]
        return self.grad_input
    

# Implement the MaxPool2D layer
class MaxPool2D:
    def __init__(self, kernel_size):
        self.kernel_size = kernel_size

    def forward(self, x):
        # Implement the forward pass for the MaxPool2D layer
        # x is the input
        # Return the output
        self.x = x
        self.output = np.zeros((x.shape[0], x.shape[1] // self.kernel_size, x.shape[2] // self.kernel_size))
        for i in range(self.x.shape[0]):
            for j in range(self.x.shape[1] // self.kernel_size):
                for k in range(self.x.shape[2] // self.kernel_size):
                    self.output[i][j][k] = np.max(self.x[i][j * self.kernel_size:(j + 1) * self.kernel_size, k * self.kernel_size:(k + 1) * self.kernel_size])
        return self.output

    def backward(self, grad_output):
        # Implement the backward pass for the MaxPool2D layer
        # grad_output is the gradient from the previous layer
        # Return the gradient for the current layer
        self.grad_input = np.zeros((self.x.shape[0], self.x.shape[1], self.x.shape[2]))
        for i in range(self.x.shape[0]):
            for j in range(self.x.shape[1] // self.kernel_size):
                for k in range(self.x.shape[2] // self.kernel_size):
                    for l in range(self.kernel_size):
                        for m in range(self.kernel_size):
                            if self.x[i][j * self.kernel_size + l][k * self.kernel_size + m] == np.max(self.x[i][j * self.kernel_size:(j + 1) * self.kernel_size, k * self.kernel_size:(k + 1) * self.kernel_size]):
                                self.grad_input[i][j * self.kernel_size + l][k * self.kernel_size + m] = grad_output[i][j][k]
        return self.grad_input
    

# Implement the Linear layer
class Linear:
    def __init__(self, in_features, out_features):
        self.in_features = in_features
        self.out_features = out_features

        # Initialize the weights
        self.weights = np.random.randn(out_features, in_features)
        self.bias = np.random.randn(out_features, 1)

    def forward(self, x):
        # Implement the forward pass for the Linear layer
        # x is the input
        # Return the output
        self.x = x
        self.output = np.zeros((self.out_features, 1))
        for i in range(self.out_features):
            self.output[i] = np.sum(self.weights[i] * self.x) + self.bias[i]
        return self.output

    def backward(self, grad_output):
        # Implement the backward pass for the Linear layer
        # grad_output is the gradient from the previous layer
        # Return the gradient for the current layer
        self.grad_weights = np.zeros((self.out_features, self.in_features))
        self.grad_bias = np.zeros((self.out_features, 1))
        self.grad_input = np.zeros((self.in_features, 1))
        for i in range(self.out_features):
            self.grad_weights[i] = grad_output[i] * self.x
            self.grad_bias[i] = grad_output[i]
            self.grad_input += grad_output[i] * self.weights[i]
        return self.grad_input
    

# Implement the ReLU layer
class ReLU:
    def __init__(self):
        pass

    def forward(self, x):
        # Implement the forward pass for the ReLU layer
        # x is the input
        # Return the output
        self.x = x
        self.output = np.zeros(x.shape)
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                for k in range(x.shape[2]):
                    self.output[i][j][k] = max(0, x[i][j][k])
        return self.output

    def backward(self, grad_output):
        # Implement the backward pass for the ReLU layer
        # grad_output is the gradient from the previous layer
        # Return the gradient for the current layer
        self.grad_input = np.zeros(self.x.shape)
        for i in range(self.x.shape[0]):
            for j in range(self.x.shape[1]):
                for k in range(self.x.shape[2]):
                    if self.x[i][j][k] > 0:
                        self.grad_input[i][j][k] = grad_output[i][j][k]
        return self.grad_input

In [ ]:
# Implement CONV1: Kernel size (3 × 3), In channels 3, Out channels 32.
conv1 = Conv2D(3, 32, 3)
# Implement POOL1: Kernel size (2 × 2)
pool1 = MaxPool2D(2)
# Implement CONV2: Kernel size (5 × 5), In channels 32, Out channels 64
conv2 = Conv2D(32, 64, 5)
# Implement POOL2: Kernel size (2 × 2)
pool2 = MaxPool2D(2)
# Implement CONV3: Kernel size (3 × 3), In channels 64, Out channels 64.
conv3 = Conv2D(64, 64, 3)
# Implement FC1: Fully connected layer (also known as Linear layer) with 64 output neurons.
fc1 = Linear(64, 64)
# Implement FC2: Fully connected layer with 10 output neurons.
fc2 = Linear(64, 10)

# Use the categorical cross entropy loss as the loss function.
def loss_function(y, y_hat):
    # Implement the loss function
    # y is the ground truth label
    # y_hat is the output from the forward pass
    # Return the loss
    loss = 0
    for i in range(y.shape[0]):
        loss += -np.log(y_hat[i][y[i]])
    return loss

# Implement the activation function RELU where x has shape (a, b)
def relu(x):
    # Implement the RELU function
    # Return the output
    output = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            for k in range(x.shape[2]):
                 output[i][j][k] = max(0, x[i][j][k])
    return output

# Apply RELU activation function to the output of the CONV1, CONV2, and CONV3 layers and the FC1 layer.

# Implement the forward pass
def forward(x):
    # Implement the forward pass
    # x is the input
    # Return the output
    x = conv1.forward(x)
    x = relu(x)
    x = pool1.forward(x)
    # x = np.expand_dims(x,axis=3)
    x = conv2.forward(x)
    x = relu(x)
    x = pool2.forward(x)
    x = conv3.forward(x)
    x = relu(x)
    x = x.reshape((x.shape[0], x.shape[3]))
    x = fc1.forward(x)
    x = relu(x)
    x = fc2.forward(x)
    return x

# Implement the backward pass
def backward(y, y_hat):
    # Implement the backward pass
    # y is the ground truth label
    # y_hat is the output from the forward pass
    # Return the gradients for the parameters
    grad_output = np.zeros((y_hat.shape[0], y_hat.shape[1]))
    for i in range(y.shape[0]):
        grad_output[i][y[i]] = -1 / y_hat[i][y[i]]
    grad_output = fc2.backward(grad_output)
    grad_output = relu.backward(grad_output)
    grad_output = fc1.backward(grad_output)
    grad_output = relu.backward(grad_output)
    grad_output = grad_output.reshape((grad_output.shape[0], 1, 1, grad_output.shape[1]))
    grad_output = conv3.backward(grad_output)
    grad_output = relu.backward(grad_output)
    grad_output = pool2.backward(grad_output)
    grad_output = conv2.backward(grad_output)
    grad_output = relu.backward(grad_output)
    grad_output = pool1.backward(grad_output)
    grad_output = conv1.backward(grad_output)
    return grad_output

# Implement the Adam optimizer
class Adam:
    def __init__(self, parameters, learning_rate, beta1=0.9, beta2=0.999, epsilon=1e-8):
        # parameters is a list of parameters to optimize
        # learning_rate is the learning rate
        # beta1, beta2, epsilon are hyperparameters of the Adam optimizer
        self.parameters = parameters
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = [np.zeros(p.shape) for p in parameters]
        self.v = [np.zeros(p.shape) for p in parameters]
        self.t = 0

    def step(self, gradients):
        # Implement the Adam optimizer
        # gradients is a list of gradients for the parameters
        # Return the updated parameters
        self.t += 1
        for i in range(len(self.parameters)):
            self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * gradients[i]
            self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * gradients[i] * gradients[i]
            m_hat = self.m[i] / (1 - self.beta1 ** self.t)
            v_hat = self.v[i] / (1 - self.beta2 ** self.t)
            self.parameters[i] -= self.learning_rate * m_hat / (np.sqrt(v_hat) + self.epsilon)
        return self.parameters


# Complete the training code, and train for 20 epochs using an Adam optimizer with learning rate 0.001 and batch size 32.

# Using the Adam class, train the data 

def train(x, y, model, loss_function, optimizer, batch_size, epochs):
    # x is the training data
    # y is the training labels
    # model is the model to train
    # loss_function is the loss function
    # optimizer is the optimizer
    # batch_size is the batch size
    # epochs is the number of epochs
    # Return the trained model
    for epoch in range(epochs):
        print('Epoch: ', epoch + 1)
        for i in range(0, x.shape[0], batch_size):
            x_batch = x[i:i + batch_size]
            y_batch = y[i:i + batch_size]
            y_hat = forward(x_batch)
            loss = loss_function(y_batch, y_hat)
            print('Loss: ', loss)
            gradients = backward(y_batch, y_hat)
            optimizer.step(gradients)
    return model




In [ ]:
import os 
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["MKL_NUM_THREADS"] = "4"

# model = train(data[0], labels, [conv1, conv2, conv3, fc1, fc2], loss_function, Adam([conv1.weights, conv1.bias, conv2.weights, conv2.bias, conv3.weights, conv3.bias, fc1.weights, fc1.bias, fc2.weights, fc2.bias], 0.001), batch_size=32, epochs=20)
# Train the model for one image

model = train(data, labels, [conv1, conv2, conv3, fc1, fc2], loss_function, Adam([conv1.weights, conv1.bias, conv2.weights, conv2.bias, conv3.weights, conv3.bias, fc1.weights, fc1.bias, fc2.weights, fc2.bias], 0.001), batch_size=32, epochs=20)